# Assignment 2

## Angelo Rosace



In [ ]:
import pandas as pd
import seaborn as sn
import random as rand
import matplotlib.pyplot as plt 

### Assignment 2: exercise 1

First of all we will need to read the file containing the result of the experiment

In [ ]:
diet_and_atp = pd.read_csv("diet_and_atp.tsv", sep = "\t", header = [0])
diet_and_atp

Above is the table containing the data that came from the experiment done in the lab.
In order to have a deeper understanding of the data we will plot each of the distributions of the different values.

In [ ]:
atp = diet_and_atp["ATP"]
prot = diet_and_atp["Protein"]
carb = diet_and_atp["Carbohydrates"]
fat = diet_and_atp["Fat"]

plt.subplot(2,2,1)
sn.distplot(atp, label="ATP")
plt.legend()
plt.subplot(2,2,2)
sn.distplot(prot, label="Proteins")
plt.legend()
plt.subplot(2,2,3)
sn.distplot(carb, label="Carbohidrates")
plt.legend()
plt.subplot(2,2,4)
sn.distplot(fat, label="Fats")
plt.legend()
plt.show

By taking a look to the graphs for all the variables individually we can recognize some sort of normal distribution for all of them.

Besides from the data the experiment tells us also that the model describing our data is defined by this function:
**atp = a * Protein + b * Carbohidrates + c * Fat + ε**

This information plus the information we get by inspecting the data leads us to the next step: 
Implement a Markov Chain Monte Carlo algorithm.

In [ ]:
def mean(lst): 
    return sum(lst) / len(lst)

# I want to obtain the posterior distribution of my samples

iterations = 100000
# This are the initial guesses for the mean. They are arbitrary, changing them will affect the execution of the algorithm
current_prot = mean(prot) #- 0.5
current_carb = mean(carb) #- 1
current_fat = mean(fat) #- 0.05

# Record of the steps. It is a list of lists containing a list for each iteration. This lists are made of the values of the three variables explaining the model
all_steps_prot = []
all_steps_carb = []
all_steps_fat = []
# Record of the posterior distribution
all_currents_prot = []
all_currents_carb = []
all_currents_fat = []

for i in range(iterations):
    # Move randomly, forward or backwards, at a maximun distance of 0.5 in each direction, that is on the surfaces explained by the three variables
    # The different distances for the different variables are due to the scale of each one of them, the larger the scale the longer the distance.
    step_prot = (rand.random()/10) - 0.05
    step_carb = rand.random() - 0.5
    step_fat = (rand.random()/10)-0.05
    # The step that I will decide if I accept it or not
    candidate_current_prot = current_prot + step_prot
    candidate_current_carb = current_carb + step_carb
    candidate_current_fat = current_fat + step_fat
    
    p_actual_step_prot = len([sample for sample in prot if current_prot - 0.05 < sample and sample < current_prot + 0.05]) / float(len(prot))
    p_candidate_step_prot = len([sample for sample in prot if candidate_current_prot - 0.05 < sample and sample < candidate_current_prot + 0.05]) / float(len(prot))
    
    p_actual_step_carb = len([sample for sample in carb if current_carb - 0.5 < sample and sample < current_carb + 0.5]) / float(len(carb))
    p_candidate_step_carb = len([sample for sample in carb if candidate_current_carb - 0.5 < sample and sample < candidate_current_carb + 0.5]) / float(len(carb))
    
    p_actual_step_fat = len([sample for sample in fat if current_fat - 0.05 < sample and sample < current_fat + 0.05]) / float(len(fat))
    p_candidate_step_fat = len([sample for sample in fat if candidate_current_fat - 0.05 < sample and sample < candidate_current_fat + 0.05]) / float(len(fat))
    
    
    # If the new step is better, I always accept
    # If not, I accept it randomly
    if p_candidate_step_prot > p_actual_step_prot:
        current_prot = candidate_current_prot
    else:
        random_number = rand.random()
        if random_number <= p_candidate_step_prot / p_actual_step_prot:
            current_prot = candidate_current_prot
    
    if p_candidate_step_carb > p_actual_step_carb:
        current_carb = candidate_current_carb
    else:
        random_number = rand.random()
        if random_number <= p_candidate_step_carb / p_actual_step_carb:
            current_carb = candidate_current_carb
            
    if p_candidate_step_fat > p_actual_step_fat:
        current_fat = candidate_current_fat
    else:
        random_number = rand.random()
        if random_number <= p_candidate_step_fat / p_actual_step_fat:
            current_fat = candidate_current_fat
    
    
    # I record my step and where I'm standing
    all_currents_prot.append(current_prot)
    all_currents_carb.append(current_carb)
    all_currents_fat.append(current_fat)
    all_steps_prot.append(step_prot)
    all_steps_carb.append(step_carb)
    all_steps_fat.append(step_fat)
    
    

# If I plot all my standings, I will have the likelihood function of the mean of the data 
# (the parameter of the model that I'm inferring)
fig=plt.figure(figsize=(12, 12))
sn.distplot(all_currents_prot, kde=False, label="my guess after %s iterations" % iterations, norm_hist=True)
# And I compare them with the original samples
sn.distplot(prot, kde=False, label="original sample", norm_hist=True)
plt.legend()

fig2=plt.figure(figsize=(12, 12))
sn.distplot(all_currents_carb, kde=False, label="my guess after %s iterations" % iterations, norm_hist=True)
sn.distplot(carb, kde=False, label="original sample", norm_hist=True)
plt.legend()

fig3=plt.figure(figsize=(12, 12))
sn.distplot(all_currents_fat, kde=False, label="my guess after %s iterations" % iterations, norm_hist=True)
sn.distplot(fat, kde=False, label="original sample", norm_hist=True)
plt.legend()

In [ ]:
prot_coeficient = mean(all_currents_prot)
carb_coeficient = mean(all_currents_carb)
fat_coeficient = mean(all_currents_fat)

print("The inferred coeficient for the protein variable of the model is:", prot_coeficient)
print("The inferred coeficient for the carbohidrate variable of the model is:", carb_coeficient)
print("The inferred coeficient for the fat variable of the model is:", fat_coeficient)

Having inferred the coeficients of the variable, the only thing that is left to do is to consider and infer the error that the model describes.

In order to do so I will consider the standard errors of all the distributions and sum them. 

In [ ]:
#calculate standard error
def standard_error(lst):
    mu = mean(lst)
    var = 0
    n = len(lst)
    for i in lst:
        var = var + ((i-mu)**2)
    sd = (var/(n-1))**0.5
    return sd/(n**0.5)
    
overall_error = standard_error(all_currents_prot) + standard_error(all_currents_carb) + standard_error(all_currents_fat)
print("The overall standard error for our posterior distributions is: ", overall_error)

### Assignment 1: Exercise 5

Compute the probabilities that you already computed by hand by counting the outcomes describing each roll result in the list of all possible hands.


In [1]:
allRolls5Dice = lambda : [[a+1,b+1,c+1,d+1,e+1] for a in range (6)\
                                                for b in range (6)\
                                                for c in range (6)\
                                                for d in range (6)\
                                                for e in range (6)]

allRollsNDices = lambda n: [[a+1]+smallerRoll for a in range(6) for smallerRoll in allRollsNDices(n-1)] if n > 1 else [[a+1] for a in range(6)]

isGenerala = lambda roll: True if len(set(roll)) == 1 else False

p_served_generala = len([roll for roll in allRolls5Dice() if isGenerala(roll)]) / float(len(allRolls5Dice()))

notServedGeneralas = [roll for roll in allRolls5Dice() if not isGenerala(roll)]

waysOfChooseFromFiveDice = 5 + 5*4 + 5*4*3 + 5*4*3*2 + 5*4*3*2*1

In [2]:
unUnsefulPickUpsCount = 0
usefulPickUps = []
for notServedGenerala in notServedGeneralas:
    for diceType in set(notServedGenerala):
        usefulPickUp = [dice for dice in notServedGenerala if dice == diceType]
        usefulPickUps.append(usefulPickUp)
        
        unUnsefulPickUpsCount += waysOfChooseFromFiveDice - len(usefulPickUp)
        
#print (len (usefulPickUps), unUnsefulPickUpsCount)

27900 9028650


In [3]:
generalaCountAfterReroll = 0
notGeneralaCountAfterReroll = 0
#notGeneralasAfterReroll = []
for usefulPickUp in usefulPickUps:
    for reroll in allRollsNDices(5-len(usefulPickUp)):
        if isGenerala(usefulPickUp+reroll): generalaCountAfterReroll += 1
        else: 
            notGeneralaCountAfterReroll += 1
            #notGeneralasAfterReroll.append(usefulPickUp+reroll)
        
#print (generalaCountAfterReroll, notGeneralaCountAfterReroll)

p_generala_after_two_rolls = p_served_generala + ( generalaCountAfterReroll / (generalaCountAfterReroll+notGeneralaCountAfterReroll + unUnsefulPickUpsCount) )

27900 25947000


In [12]:
generalaCountAfterLastReroll = 0
notGeneralaCountAfterLastReroll = notGeneralaCountAfterReroll
for usefulLastPickUp in usefulPickUps:
    for reroll in allRollsNDices(5-len(usefulPickUp)):
        if isGenerala(usefulLastPickUp+reroll): generalaCountAfterLastReroll += 1
        else: 
            notGeneralaCountAfterLastReroll += 1

#print (generalaCountAfterLastReroll, notGeneralaCountAfterLastReroll)
p_generala_after_three_rolls = p_generala_after_two_rolls + ( generalaCountAfterLastReroll / (generalaCountAfterLastReroll+notGeneralaCountAfterLastReroll + unUnsefulPickUpsCount) )
print("the prbability of getting a generala after three rerolls is :", p_generala_after_three_rolls)

the prbability of getting a generala after three rerolls is : 0.002362565032819838
